# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f59d833f1f0>
├── 'a' --> tensor([[ 0.3114, -2.1245,  0.1693],
│                   [-0.8827,  0.1116,  0.7627]])
└── 'x' --> <FastTreeValue 0x7f59d833f130>
    └── 'c' --> tensor([[ 0.0824, -1.2746, -1.0552,  1.7220],
                        [-1.1312, -0.2202, -0.0143,  1.4858],
                        [ 0.7722, -0.5205, -0.5121, -0.3885]])

In [4]:
t.a

tensor([[ 0.3114, -2.1245,  0.1693],
        [-0.8827,  0.1116,  0.7627]])

In [5]:
%timeit t.a

85.2 ns ± 0.877 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f59d833f1f0>
├── 'a' --> tensor([[-0.2215,  0.1619, -0.1763],
│                   [-0.7978, -0.0673,  1.4471]])
└── 'x' --> <FastTreeValue 0x7f59d833f130>
    └── 'c' --> tensor([[ 0.0824, -1.2746, -1.0552,  1.7220],
                        [-1.1312, -0.2202, -0.0143,  1.4858],
                        [ 0.7722, -0.5205, -0.5121, -0.3885]])

In [7]:
%timeit t.a = new_value

101 ns ± 0.896 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.3114, -2.1245,  0.1693],
               [-0.8827,  0.1116,  0.7627]]),
    x: Batch(
           c: tensor([[ 0.0824, -1.2746, -1.0552,  1.7220],
                      [-1.1312, -0.2202, -0.0143,  1.4858],
                      [ 0.7722, -0.5205, -0.5121, -0.3885]]),
       ),
)

In [10]:
b.a

tensor([[ 0.3114, -2.1245,  0.1693],
        [-0.8827,  0.1116,  0.7627]])

In [11]:
%timeit b.a

81.9 ns ± 0.482 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.7063, -0.2875, -0.4860],
               [ 0.3716, -0.5215,  0.3639]]),
    x: Batch(
           c: tensor([[ 0.0824, -1.2746, -1.0552,  1.7220],
                      [-1.1312, -0.2202, -0.0143,  1.4858],
                      [ 0.7722, -0.5205, -0.5121, -0.3885]]),
       ),
)

In [13]:
%timeit b.a = new_value

681 ns ± 13.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.14 µs ± 6.69 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

14.2 µs ± 107 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

292 µs ± 19.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

274 µs ± 15.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f590a95c070>
├── 'a' --> tensor([[[ 0.3114, -2.1245,  0.1693],
│                    [-0.8827,  0.1116,  0.7627]],
│           
│                   [[ 0.3114, -2.1245,  0.1693],
│                    [-0.8827,  0.1116,  0.7627]],
│           
│                   [[ 0.3114, -2.1245,  0.1693],
│                    [-0.8827,  0.1116,  0.7627]],
│           
│                   [[ 0.3114, -2.1245,  0.1693],
│                    [-0.8827,  0.1116,  0.7627]],
│           
│                   [[ 0.3114, -2.1245,  0.1693],
│                    [-0.8827,  0.1116,  0.7627]],
│           
│                   [[ 0.3114, -2.1245,  0.1693],
│                    [-0.8827,  0.1116,  0.7627]],
│           
│                   [[ 0.3114, -2.1245,  0.1693],
│                    [-0.8827,  0.1116,  0.7627]],
│           
│                   [[ 0.3114, -2.1245,  0.1693],
│                    [-0.8827,  0.1116,  0.7627]]])
└── 'x' --> <FastTreeValue 0x7f590a55c250>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

53.1 µs ± 2.44 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f590a55c730>
├── 'a' --> tensor([[ 0.3114, -2.1245,  0.1693],
│                   [-0.8827,  0.1116,  0.7627],
│                   [ 0.3114, -2.1245,  0.1693],
│                   [-0.8827,  0.1116,  0.7627],
│                   [ 0.3114, -2.1245,  0.1693],
│                   [-0.8827,  0.1116,  0.7627],
│                   [ 0.3114, -2.1245,  0.1693],
│                   [-0.8827,  0.1116,  0.7627],
│                   [ 0.3114, -2.1245,  0.1693],
│                   [-0.8827,  0.1116,  0.7627],
│                   [ 0.3114, -2.1245,  0.1693],
│                   [-0.8827,  0.1116,  0.7627],
│                   [ 0.3114, -2.1245,  0.1693],
│                   [-0.8827,  0.1116,  0.7627],
│                   [ 0.3114, -2.1245,  0.1693],
│                   [-0.8827,  0.1116,  0.7627]])
└── 'x' --> <FastTreeValue 0x7f590a55c7c0>
    └── 'c' --> tensor([[ 0.0824, -1.2746, -1.0552,  1.7220],
                        [-1.1312, -0.2202, -0.0143,  1.4858],
                 

In [23]:
%timeit t_cat(trees)

47.2 µs ± 1.46 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

103 µs ± 6.4 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.0824, -1.2746, -1.0552,  1.7220],
                       [-1.1312, -0.2202, -0.0143,  1.4858],
                       [ 0.7722, -0.5205, -0.5121, -0.3885]],
              
                      [[ 0.0824, -1.2746, -1.0552,  1.7220],
                       [-1.1312, -0.2202, -0.0143,  1.4858],
                       [ 0.7722, -0.5205, -0.5121, -0.3885]],
              
                      [[ 0.0824, -1.2746, -1.0552,  1.7220],
                       [-1.1312, -0.2202, -0.0143,  1.4858],
                       [ 0.7722, -0.5205, -0.5121, -0.3885]],
              
                      [[ 0.0824, -1.2746, -1.0552,  1.7220],
                       [-1.1312, -0.2202, -0.0143,  1.4858],
                       [ 0.7722, -0.5205, -0.5121, -0.3885]],
              
                      [[ 0.0824, -1.2746, -1.0552,  1.7220],
                       [-1.1312, -0.2202, -0.0143,  1.4858],
                       [ 0.7722, -0.5205, -0.5121, -0.3885]],

In [26]:
%timeit Batch.stack(batches)

122 µs ± 2.73 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.0824, -1.2746, -1.0552,  1.7220],
                      [-1.1312, -0.2202, -0.0143,  1.4858],
                      [ 0.7722, -0.5205, -0.5121, -0.3885],
                      [ 0.0824, -1.2746, -1.0552,  1.7220],
                      [-1.1312, -0.2202, -0.0143,  1.4858],
                      [ 0.7722, -0.5205, -0.5121, -0.3885],
                      [ 0.0824, -1.2746, -1.0552,  1.7220],
                      [-1.1312, -0.2202, -0.0143,  1.4858],
                      [ 0.7722, -0.5205, -0.5121, -0.3885],
                      [ 0.0824, -1.2746, -1.0552,  1.7220],
                      [-1.1312, -0.2202, -0.0143,  1.4858],
                      [ 0.7722, -0.5205, -0.5121, -0.3885],
                      [ 0.0824, -1.2746, -1.0552,  1.7220],
                      [-1.1312, -0.2202, -0.0143,  1.4858],
                      [ 0.7722, -0.5205, -0.5121, -0.3885],
                      [ 0.0824, -1.2746, -1.0552,  1.7220],
                   

In [28]:
%timeit Batch.cat(batches)

222 µs ± 9.37 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

621 µs ± 25.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
